<h1 align="center"> Computación Cientifica - Laboratorio 5: Sistemas de Reacción-Difusión </h1>
<br>
<br>
** Autores **:
+ Yerson Escobar - 201273084-8 
+ Eduardo Ramírez - 201103031-1


## Introducción



## Desarrollo

### Diferencias en 2D

a) Sea el dominio de las funciones $ u $ y $ v $: $ [a,b] \times [c,d] \times [0, T_{max}] $. <br>
Discretizaremos el dominio de todos los dominios en $ N_x + 1 $, $ N_y + 1 $, $ N_t + 1 $, tal que $ \Delta x = \frac{b - a}{N_x} $, $ \Delta y = \frac{d - c}{N_y} $, $ \Delta t = \frac{T_{max}}{N_t} $ y $ x_i = i\Delta x + a $, $ y_j = j\Delta y + c $ y $ t_k = k\Delta t $; $ (i,j,k) \in \left\lbrace 0,1,\ldots, N_x + 1\right\rbrace \times \left\lbrace 0,1,\ldots, N_y + 1 \right\rbrace \times \left\lbrace 0,1,\ldots, N_t + 1\right\rbrace $. Usando la discretización de $ u_{xx} = \frac{u_{i-1,j,k} - 2u_{i,j,k} + u_{i+1,j,k}}{\Delta x^2} $, misma para $ u_{yy} $ y $ u_t = \frac{u_{i,j,k+1} - u_{i,j,k}}{\Delta t} $ con $ u_{i,j,k} = u(x_i,y_j,t_k) $ y definiendo $\phi_{u,x} = \frac{D_u\Delta t}{\Delta x^2}; \phi_{u,y} = \frac{D_u\Delta t}{\Delta y^2}; \phi_{v,x} = \frac{D_v \Delta t}{\Delta x^2}; \phi_{v,y} = \frac{D_v\Delta t}{\Delta y^2}$, $U_k$ y $V_k$ matrices de soluciones en el instante $k$,  $U_k\{m\}$ es la matriz de soluciones con cada coeficiente elevado a $m$ y además:

$$
A=
\left(
\begin{array}{ccccc}
 -2 & 1 & 0 & \cdots & 1 \\
 1 & -2 & 1 & \cdots & 0 \\
 \vdots & \vdots & \ddots & \vdots & \vdots \\
 0 & \cdots & 1 & -2 & 1 \\
 1 & \cdots & 0 & 1 & -2 \\
\end{array}
\right)
$$
<br> <br>

 entonces, las EDPs discretizadas tendrán la siguiente forma
$$ 
    \begin{align} 
    (1) \frac{u_{i,j,k+1} - u_{i,j,k}}{\Delta t} &= D_u \left( \frac{u_{i-1,j,k} - 2 u_{i,j,k} + u_{i+1,j,k}}{\Delta x^2} + \frac{u_{i,j-1,k} - 2u_{i,j,k} + u_{i,j+1,k}}{\Delta y^2} \right) + \lambda u_{i,j,k} - u^3_{i,j,k} - \kappa - \sigma v_{i,j,k}\\
    u_{i,j,k+1} &= [\phi_{u,x} u_{i+1,j,k} -2\phi_{u,x}u_{i,j,k} + \phi_{u,x} u_{i-1,j,k}] + [\phi_{u,y} u_{i,j+1,k} -2\phi_{u,y}u_{i,j,k} +\phi_{u,y} u_{i,j-1,k}]+ (1 +\lambda \Delta t) u_{i,j,k}  - u^3_{i,j,k} \Delta t - \kappa \Delta t - \sigma v_{i,j,k} \Delta t \\
    U_{k+1} &= A (\phi_{u,x}U_k + \phi_{u,y}U^T_k) + (\Delta t \lambda + 1)U_k - U_k\{3\} - \Delta t\kappa I - \Delta t\sigma V_k  
\end{align} 
$$
<br>

<br>
$$ 
    \begin{align}
        (2) \tau \frac{v_{i,j,k+1} - v_{i,j,k}}{\Delta t} &= D_v \left( \frac{v_{i-1,j,k} - 2 v_{i,j,k} + v_{i+1,j,k}}{\Delta x^2} + \frac{v_{i,j-1,k} - 2v_{i,j,k} + v_{i,j+1,k}}{\Delta y^2} \right) + u_{i,j,k} - v_{i,j,k} \\
        v_{i,j,k+1} &= \frac{1}{\tau}[\phi_{v,x} v_{i+1,j,k} -2\phi_{v,x}v_{i,j,k} + \phi_{v,x} v_{i-1,j,k}] + \frac{1}{\tau}[\phi_{v,y} v_{i,j+1,k} -2\phi_{v,y}v_{i,j,k} +\phi_{v,y} v_{i,j-1,k}] + \frac{\Delta t}{\tau}u_{i,j,k} + \left(1-\frac{\Delta t}{\tau}\right)v_{i,j,k} \\
        V_{k+1} &= \frac{1}{\tau}A (\phi_{v,x}V_k + \phi_{v,y}V^T_k) +\frac{\Delta t}{\tau}U_k+ \left(\frac{1-\Delta t}{\tau}\right)V_k    
    \end{align}
$$
    
b)

In [1]:
'''
def poop_solver(u0,v0,dx,dy,dt,tf,D_u,D_v,lamb,tau,sigma,kappa):
    phiu1 = 1.0*D_u*dt/dx**2
    phiu2 = 1.0*D_u*dt/dy**2
    phiv1 = 1.0*D_v*dt/dx**2
    phiv2 = 1.0*D_v*dt/dy**2
    if phiu1 >= 0.5 or phiu2 >= 0.5 or phiv1 >= 0.5 or phiv2 >= 0.5:
        raise Exception("Inestable")
    Nx = u0.shape[0] - 1
    Ny = u0.shape[1] - 1
    ts = np.arange(0,Tmax,dt)
    Nt = ts.shape[0] - 1
    u = np.zeros([Nt+1, Nx+1, Ny+1])
    v = np.zeros([Nt+1, Nx+1, Ny+1])
    
    #pa formar las matrices con las constantes.
    matrix_phiu1 = np.zeros([Ny+1, Nx+1])
    matrix_phiu2 = np.zeros([Ny+1, Nx+1])
    matrix_phiv1 = np.zeros([Ny+1, Nx+1])
    matrix_phiv2 = np.zeros([Ny+1, Nx+1])
    
    matrix_phiu1[0,0] = 1. + 2.*phiu1
    matrix_phiu1[Ny,Nx] = matrix_phiu1[0,0]
    matrix_phiu1[0,1] = phiu1
    matrix_phiu1[Ny,Nx-1] = matrix_phiu1[0,1]
    matrix_phiu1[0,Nx] = phiu1
    matrix_phiu1[Ny,0] = matrix_phiu1[0,Nx]
    matrix_phiu1[i-1:i+2 for i in range(1,Ny-1)] = np.array([phiu1, 1. + 2.*phiu1, phiu1])
    
    matrix_phiu2[0,0] = -lamb*dt - 2.*phiu2
    matrix_phiu2[Ny,Nx] = matrix_phiu2[0,0]
    matrix_phiu2[0,1] = phiu2
    matrix_phiu2[Ny,Nx-1] = matrix_phiu2[0,1]
    matrix_phiu2[0,Nx] = phiu2
    matrix_phiu2[Ny,0] = matrix_phiu2[0,Nx]
    matrix_phiu2[i-1:i+2 for i in range(1,Ny-1)] = np.array([phiu2, -lamb*dt. - 2.*phiu2, phiu2])
    
    matrix_phiv1[0,0] = -dt - 2.*phiv1
    matrix_phiv1[Ny,Nx] = matrix_phiv1[0,0]
    matrix_phiv1[0,1] = phiv1
    matrix_phiv1[Ny,Nx-1] = matrix_phiv1[0,1]
    matrix_phiv1[0,Nx] = phiv1
    matrix_phiv1[Ny,0] = matrix_phiv1[0,Nx]
    matrix_phiv1[i-1:i+2 for i in range(1,Ny-1)] = np.array([phiv1, -dt - 2.*phiv1, phiv1])
    
    matrix_phiv2[0,0] = -1. - 2.*phiv2
    matrix_phiv2[Ny,Nx] = matrix_phiv2[0,0]
    matrix_phiv2[0,1] = phiv2
    matrix_phiv2[Ny,Nx-1] = matrix_phiv2[0,1]
    matrix_phiv2[0,Nx] = phiv2
    matrix_phiv2[Ny,0] = matrix_phiv2[0,Nx]
    matrix_phiv2[i-1:i+2 for i in range(1,Ny-1)] = np.array([phiv2, -1. - 2.*phiv2, phiv2])
    
    
    for t in ts:
        if t == 0:
            u[0] = u0
            v[0] = v0
        else:
            u[t] = np.dot(matrix_phiu1,u0) + np.dot(matrix_phiu2,)
            '''

SyntaxError: invalid syntax (<ipython-input-1-9bdb640028aa>, line 27)

In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import animation
"""
> sol must be a 3D numpy array, with dimensions (Nt,Nx,Ny)
> time_step:  miliseconds to update canvas
> frame_step: Jump step for frames to show
"""
def animate_pattern(sol, time_step=10, frame_step=20):
    #time index
    t_ind = 0
    fig = plt.figure()
    im = plt.imshow(sol[t_ind], cmap=plt.cm.winter)
    plt.xticks([]); plt.yticks([])

    #setting the number of frames
    frames = int(sol.shape[0]/frame_step)
    
    #update function
    def updatefig(t_ind):
        im = plt.imshow(sol[t_ind*frame_step], cmap=plt.cm.winter)
        return im,
    
    #animate it!
    ani = animation.FuncAnimation(fig, updatefig, frames=frames, 
                    interval=time_step, blit=True, repeat=False)
    plt.show()
    
    
def solve_forward(u0,v0,deltax,deltay,deltat,Tf,Du,Dv,Lambda,Tau,Sigma,Kappa):
    #hay que cambiar todo esto
    phiu1 = 1.0*Du*deltat/deltax**2
    phiu2 = 1.0*Du*deltat/deltay**2
    phiv1 = 1.0*Dv*deltat/(Tau*deltax**2)
    phiv2 = 1.0*Dv*deltat/(Tau*deltay**2)
    #if phiu1 >= 0.5 or phiu2 >= 0.5 or phiv1 >= 0.5 or phiv2 >= 0.5:
    #    raise Exception("Inestable")
    Nx = u0.shape[0]
    Ny = u0.shape[0]
    ts = np.arange(0,Tf+deltat,deltat)
    if(ts[-1]>Tf):
        ts = ts[:-1]
    Nt = ts.size
    u = np.zeros((Nt,Nx,Ny),dtype=np.float64)
    v = np.zeros((Nt,Nx,Ny),dtype=np.float64)
    u[0] = u0
    v[0] = v0
    #instantes,filas,columnas
    ones = np.ones((Nx,Ny))
    z = np.zeros((Nx,Ny))
    for k in range(1,Nt):
      
    return u,v
    
u0 = np.random.rand(7,7)
v0 = np.random.rand(7,7)
deltax = 0.2
deltay = 0.8
deltat = 0.04
Tf = 4
Du = 3*np.e-4
Dv = 4*np.e-3
Lambda = 1.0
Tau = 0.2
Sigma = 1.0
Kappa = -0.004
u,v = solve_forward(u0,v0,deltax,deltay,deltat,Tf,Du,Dv,Lambda,Tau,Sigma,Kappa)
print u
animate_pattern(u)

### Método de las Líneas

Al discretizar queda:
$$
\frac{\partial u_i}{\partial t}=D_u \left(\frac{u_{i-1}+u_{i+1}-2 u_i}{\text{$\Delta $x}^2}+\frac{u_{i-1}+u_{i+1}-2 u_i}{\text{$\Delta $y}^2}\right)+\text{$\lambda $u}_i-u_i^3-\sigma  v_i-\kappa
$$
$$
\tau  \frac{\partial v_i}{\partial t}=D_v \left(\frac{v_{i-1}+v_{i+1}-2 v_i}{\text{$\Delta $x}^2}+\frac{v_{i-1}+v_{i+1}-2 v_i}{\text{$\Delta $y}^2}\right)+u_i-v_i
$$
Haciendo un poco de algebra queda